In [ ]:
%run dataImport.ipynb
%run HelperFunctions.ipynb

In [ ]:
data = merge_data()
screw = data[0]
tile = data[1]
capsule = data[2]
print(len(data))

In [ ]:
print(len(screw))
print(len(tile))
print(len(capsule))

In [ ]:
# buold predictor and stuff for Autogluon
from autogluon.multimodal import MultiModalPredictor
from datetime import datetime as dt
def fit(train_dat, label_col, seed, max_time,name):
    '''
    function to predict an Image dataset with autogluon
    '''
    # measure the fitting time
    start_time = dt.now()
    model_path = path="plots_im/Autogluon/" + name + str(seed)
    # build predictor object
    predictor = MultiModalPredictor(label=label_col, path=model_path).fit(
    train_data=train_dat, # add train data
    seed = seed,
    time_limit = max_time
        # max time limit
    )
    
    #fit the model
    fit_summary = predictor.fit_summary(verbosity=0, show_plot=True) # print no output, too long for the notebook
    # end of time measure
    end_time = dt.now()
    
    elapsed_seconds = (end_time - start_time).total_seconds()
    elapsed_min = divmod(elapsed_seconds, 60)
    # print the elapsed time
    time_passed = f"{int(elapsed_min[0])}m{int(elapsed_min[1])}s"
    print("Total fitting time: ", f"{int(elapsed_min[0])}m{int(elapsed_min[1])}s")
    
    # print the fir summary
    print(fit_summary)
    return predictor, fit_summary, time_passed
    

In [ ]:
def max_times(file):
    times = []
    with open(file, 'r') as file:
            for line in file:            
                try:
                    line.replace("\n","")
                    line = int(float(line))
                    times.append(line)
                except:
                    line = int(float(line))
                    times.append(line) 
    return times

time_screw = max_times("MVTEC.txt")
time_tile = max_times("MVTEC_tile.txt")
time_capsule = max_times("MVTEC_capsule.txt")
print(time_screw)
print(time_tile)
print(time_capsule)

In [ ]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

def classify_image(dataset, times, name, metricName):
    warnings.filterwarnings('ignore')
    seeds = [1,2,3,4,5,6,7,8,9,10]
    # latex table
    from tabulate import tabulate
    table_mvtec = pd.DataFrame(columns=["Accuracy", "ROC AUC", "Precision", "F1-score", "Time"]) 
    #[["Object", "Accuracy", "ROC AUC", "Precision","f1", "Time"]]
    
    count_mvtec = 0
    
        # run experiment series 10 times
    for i in seeds:
        np.random.seed(i)
        # extract which datset
        #name = get_name(data)
        train, test = prepare_data(dataset,i)
    
        # call the fit function
        predictor, fit_summary, time = fit(train, "label", i, times[count_mvtec],name)
    
        probs = predictor.predict_proba(test.drop(columns="label"))
        y_test = test["label"].to_numpy()
        y_test = [1 if value == "good" else 0 for value in y_test]
        probs = probs.to_numpy() 
        y_pred = np.argmax(probs ,axis=1)
    
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap="Greens", cbar=True,
                xticklabels=['0', '1'], 
                yticklabels=['0', '1'])
        plt.xlabel('Predicted label')
        plt.ylabel('True label')
        plt.savefig("plots_im/Autogluon/" + name + str(count_mvtec))
    
    
        eval = predictor.evaluate(test, metrics=["accuracy", "roc_auc", "precision", "f1"])
        row_f = {"Accuracy": eval["accuracy"], 'ROC AUC': eval["roc_auc"], 'Precision': eval["precision"], "F1-score":eval["f1"],"Time":time}
        table_mvtec.loc[len(table_mvtec)] = row_f
        count_mvtec += 1
    
    table_mvtec.to_csv(metricName, index = True)

In [ ]:
classify_image(screw,screw_tile, "screw", "Autogluon_metrics_screw.csv")

In [ ]:
classify_image(tile,time_tile, "tile", "Autogluon_metrics_tile.csv")

In [ ]:
classify_image(capsule,time_capsule, "capsule", "Autogluon_metrics_capsule.csv")